<a href="https://colab.research.google.com/github/Dan-Blanchette/DS_SP2025_Team_CDA/blob/main/Data_Science_AQI_Asthma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science: AQI and Asthma Correlation

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("merged_data.csv")

# Part 2.1 (2A)

# Part 2.2(2B)

## Dan's Model
Model 1: VGboosting + Random Forest manual stacking for AQI threhold predictions and county locations that are forecasted to have the most hospitilizations(above 50%).

In [ ]:
# Author: Dan Blanchette
# Credit: sklearn documentation, plotly documentation, US Census Bureau,
# and ChatGPT for help with geopandas heatmap.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.base import clone
import joblib



# --- Load and clean data ---
# Load the preprocessed dataset
df = pd.read_csv("/content/cleaned_aqi_hospitalizations.csv")

# --- Metrics ---
# Calculate and print total actual hospitalizations in the test set
total_hospitalizations = test_df['Value'].sum()
print(f"Total Actual Hospitalizations in Test Set: {total_hospitalizations:,.0f}")

# Remove unnecessary columns and drop rows with missing target values
df_clean = df.drop(columns=['Data Comment', 'Unnamed: 7'], errors='ignore')
df_clean = df_clean.dropna(subset=['Value'])
df_clean['Value'] = pd.to_numeric(df_clean['Value'], errors='coerce')
df_clean = df_clean.dropna(subset=['Value'])

# --- Feature engineering ---
# Add derived features: ratio of unhealthy days and combined pollutant load
df_clean['Unhealthy_Day_Ratio'] = df_clean['Unhealthy Days'] / df_clean['Days with AQI']
df_clean['Pollutant_Load'] = df_clean['Days PM2.5'] + df_clean['Days PM10']

# --- Filter for high-impact counties ---
# Identify counties with above-median average hospitalization values
county_avg = df_clean.groupby('CountyFIPS')['Value'].mean()
high_impact_fips = county_avg[county_avg > county_avg.median()].index
filtered_df = df_clean[df_clean['CountyFIPS'].isin(high_impact_fips)]

# --- Define feature columns ---
# These are the AQI and engineered features to be used for prediction
aqi_features = [
    'Days with AQI', 'Good Days', 'Moderate Days',
    'Unhealthy for Sensitive Groups Days', 'Unhealthy Days',
    'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
    '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2',
    'Days Ozone', 'Days PM2.5', 'Days PM10',
    'Unhealthy_Day_Ratio', 'Pollutant_Load', 'Year'
]

# Split into features (X) and target (y)
X = filtered_df[aqi_features]
y = filtered_df['Value'].values.reshape(-1, 1)

# --- Scale features and target ---
# Normalize the feature and target values to a 0-1 range
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y).ravel()

# --- Manual stacking using TimeSeriesSplit ---
# Generate out-of-fold predictions for each base model manually


tscv = TimeSeriesSplit(n_splits=5)

# Placeholders for meta-model training data
meta_features = np.zeros((X_scaled.shape[0], 2))
meta_targets = np.zeros(X_scaled.shape[0])

# Initialize base models
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Collect out-of-fold predictions from base models
for fold, (train_idx, val_idx) in enumerate(tscv.split(X_scaled)):
    print(f"Training fold {fold + 1}...")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

    # Clone and train models to avoid data leakage
    xgb_model = clone(xgb).fit(X_train, y_train)
    rf_model = clone(rf).fit(X_train, y_train)

    # Store predictions for meta-model training
    meta_features[val_idx, 0] = xgb_model.predict(X_val)
    meta_features[val_idx, 1] = rf_model.predict(X_val)
    meta_targets[val_idx] = y_val

# Train the meta-model on stacked predictions
meta_model = Ridge()
meta_model.fit(meta_features, meta_targets)

# --- Final base model training on full dataset ---
xgb.fit(X_scaled, y_scaled)
rf.fit(X_scaled, y_scaled)

# --- Predict on final holdout (2020+) ---
test_df = filtered_df[filtered_df["Year"] >= 2020]
X_test = scaler.transform(test_df[aqi_features])
y_test = test_df['Value'].values.reshape(-1, 1)
y_test_scaled = y_scaler.transform(y_test)

# Predict with base models and pass to meta-model
xgb_preds = xgb.predict(X_test)
rf_preds = rf.predict(X_test)
stacked_preds_scaled = meta_model.predict(np.column_stack([xgb_preds, rf_preds])).reshape(-1, 1)

# Inverse transform predictions to original scale
preds = y_scaler.inverse_transform(stacked_preds_scaled)
actual = y_scaler.inverse_transform(y_test_scaled)

# --- Metrics ---
# Print model performance: MAE and R^2
mae = mean_absolute_error(actual, preds)
r2 = r2_score(actual, preds)
print(f"\nMAE: {mae:.2f}")
print(f"R² Score: {r2:.4f}")

# --- County-level predictions ---
# Aggregate predictions at the county level and display top/bottom 10
county_preds = test_df[['CountyFIPS', 'County']].copy()
county_preds['Predicted_Hospitalizations'] = preds.flatten()
full_county_results = county_preds.groupby(['CountyFIPS', 'County']).mean().sort_values(by='Predicted_Hospitalizations', ascending=False)

print("\nTop 10 counties by predicted hospitalizations:")
print(full_county_results.head(10))
print("\nBottom 10 counties by predicted hospitalizations:")
print(full_county_results.tail(10))

# --- Save to CSV ---
# Save the county-level predictions for external use
full_county_results.to_csv("county_predictions.csv")
print("\nCounty-level predictions saved to 'county_predictions.csv'")

# --- Plot predictions vs actual ---
# Visual check: scatter plot of predicted vs actual hospitalizations
plt.figure(figsize=(8, 6))
plt.scatter(actual, preds, alpha=0.5)
plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--')
plt.xlabel("Actual Hospitalizations")
plt.ylabel("Predicted Hospitalizations")
plt.title("Actual vs Predicted Hospitalizations")
plt.grid(True)
plt.show()

# --- County map ---
# Load shapefile and merge predictions for geographic visualization
shapefile_path = "/content/tl_2023_us_county.shp"
counties = gpd.read_file(shapefile_path)

# Create a CountyFIPS identifier for merging
if {'STATEFP', 'COUNTYFP'}.issubset(counties.columns):
    counties['CountyFIPS'] = (counties['STATEFP'] + counties['COUNTYFP']).astype(int)
elif 'GEOID' in counties.columns:
    counties['CountyFIPS'] = counties['GEOID'].astype(int)
else:
    raise KeyError(f"Shapefile must contain 'STATEFP' and 'COUNTYFP', or 'GEOID'. Found columns: {list(counties.columns)}")

# Merge the predictions with the shapefile
map_df = counties.merge(full_county_results.reset_index(), on='CountyFIPS', how='left')

# Plot the map
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
map_df.plot(column='Predicted_Hospitalizations', cmap='OrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
ax.set_title("Predicted Hospitalizations by County", fontsize=16)
ax.axis('off')
plt.show()

# --- SHAP feature importance ---
# Use SHAP to explain XGBoost model feature contributions
import shap
explainer = shap.Explainer(xgb, X_scaled)
shap_values = explainer(X_test)

plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, features=X_test, feature_names=aqi_features)

# --- Save model and scalers ---
# Save the trained models and scalers for future use
joblib.dump(meta_model, "stacked_meta_model.pkl")
joblib.dump(xgb, "xgb_model.pkl")
joblib.dump(rf, "rf_model.pkl")
joblib.dump(scaler, "feature_scaler.pkl")
joblib.dump(y_scaler, "target_scaler.pkl")
print("\nModels and scalers saved successfully.")


# Evaluation:

The model is doing a great job overall, especially in the low-to-mid range values. It explains 73% of what's driving hospitalizations, with only moderate average error. There's room to improve accuracy on the higher end, but for the most part, this is a very solid, trustworthy model.

# Part 2.2 (2B) Jordan's Model

# Part 2.3(2C) Model Output Analysis